In [2]:
from dataclasses import dataclass, field
from __future__ import annotations

In [ ]:
class Calculator:
    def __init__(self) -> None:
        # Ordered according to BIDMAS
        self.operators: dict[str, callable] = {
            "^": lambda x, y: x ** y,
            "/": lambda x, y: x / y,
            "//": lambda x, y: x // y,
            "%": lambda x, y: x % y,
            "*": lambda x, y: x * y,
            "+": lambda x, y: x + y,
            "-": lambda x, y: x - y,
        }
        return

    def contains_operator(self, expression: str) -> dict[str, bool | str]:

        for operator, _ in self.operators.items():
            if operator in expression:
                return {
                    "operator_found": True,
                    "operator": operator
                }
        return {
            "operator_found": False,
            "operator": ""
        }

    def split(self, expression: str) -> str:

        expression_data = self.contains_operator(expression)
        if not expression_data[0]:
            return expression

        operator_index = expression.index(expression_data[1])
        left_hand_side: str = expression[:operator_index]
        right_hand_side: str = expression[operator_index+1:]

        return {
            "x": left_hand_side,
            "y": right_hand_side,
            "operator": expression_data[1],
        }

    def evaluate(self, expression: str) -> float:
        if expression[0].lower() == "f":
            return


calc = Calculator()

expression = "1+1+1"


calc.evaluate(expression=expression)

('1',
 {'x': '1',
  'y': '1',
  'operator': '+',
  'operation': <function __main__.Calculator.contains_operator.<locals>.<lambda>(x, y)>})

In [3]:
# Ordered according to BIDMAS
OPERATORS: dict[str, callable] = {
    "^": lambda x, y: x ** y,
    "/": lambda x, y: x / y,
    "//": lambda x, y: x // y,
    "%": lambda x, y: x % y,
    "*": lambda x, y: x * y,
    "+": lambda x, y: x + y,
    "-": lambda x, y: x - y,
}

In [ ]:
@dataclass
class Node:
    expression: str = field(default="")
    parent_node: Node = field(default=None)

    value: float = field(default=None, init=False)
    operator: str = field(default="", init=False)
    operation: callable = field(default=None, init=False)
    node_a: Node = field(default=None, init=False)
    node_b: Node = field(default=None, init=False)

    def __str__(self) -> str:
        return f"<Node: {self.expression}>"

    def __repr__(self) -> str:
        return f"<Node: {self.expression}>"

    @property
    def children(self) -> tuple[Node]:
        return (self.node_a, self.node_b)

    def set_operator(self, operator: str) -> None:
        if not operator in OPERATORS.keys():
            raise NameError(f"\"{operator}\" operator not defined")

        self.operator = operator
        self.operation = OPERATORS.get(operator)

    @property
    def is_numeric(self) -> bool:
        try:
            self.value = float(self.expression)
            return True
        except ValueError:
            return False
        except Exception as e:
            raise e

In [85]:
@dataclass
class Tree:
    initial_expression: str = field(repr=False)
    nodes: list = field(default_factory=list, init=False)

    def __post_init__(self) -> None:
        self.nodes.append(Node(self.initial_expression))
        self.propagate_single_node(self.nodes[0])

    def __getitem__(self, key) -> Node:
        return self.nodes[key]

    def __setitem__(self, key, value):
        self.nodes[key] = value

    def __delitem__(self, key):
        del self.nodes[key]

    def __repr__(self) -> str:
        def traverse(node: Node) -> str:
            if not node.children:
                return f"{node.value}"
            children = [traverse(child) for child in node.children]
            return f"{node.value} (\n{', '.join(children)}\n)"

        return traverse(self.root)

    @property
    def root(self) -> Node:
        return self.nodes[0]

    def contained_operator(self, expression: str) -> str:
        for operator, _ in OPERATORS.items():
            if operator in expression:
                return operator
        return ""

    def propagate_single_node(self, prev_node: Node) -> str:
        expression = prev_node.expression
        operator = self.contained_operator(expression)

        operator_index = expression.index(operator)
        side_a: str = expression[:operator_index]
        side_b: str = expression[operator_index+1:]

        new_node_a = Node(side_a, prev_node)
        new_node_b = Node(side_b, prev_node)
        prev_node.set_operator(operator)
        prev_node.node_a = new_node_a
        prev_node.node_b = new_node_b

        self.nodes.append(new_node_a)
        self.nodes.append(new_node_b)

    def propagate_branch_a(self, initial_node: Node) -> Node:
        current_node = initial_node
        while not current_node.is_numeric:
            self.propagate_single_node(current_node)
            current_node = current_node.node_a
        self.propagate_branch_b(current_node.parent_node)

    def propagate_branch_b(self, initial_node: Node) -> None:
        current_node = initial_node
        while not current_node.is_numeric:
            self.propagate_single_node(current_node)
            current_node = current_node.node_b

    def propagate_tree(self) -> None:
        current_node = self.root
        self.propagate_branch_a(current_node)

        return

In [89]:
tree = Tree("1+1+1")
[node.children for node in tree.nodes]

[(<Node: 1>, <Node: 1+1>), (None, None), (None, None)]